# Lie Theory for State Estimation in Robotics

## Introduction to an Optimization Problem on SO(3)

We have robot orientation at time $t$ as $R^{nb}_t$, and given $R^T R = \mathcal{I}$:
$$
\begin{align}
d\frac{R_t}{dt}R_t^T + R_td\frac{R_t^T}{dt} &= 0 \\
d\frac{R_t}{dt}R_t^T + (d\frac{R_t}{dt}R_t^T)^T &= 0 \\
\end{align}
$$
Define $S_t = d\frac{R_t}{dt}R_t^T$, then we have $S_t+ S_t^T = 0$. So $S_t$ is a skew matrix as:
$$
S_t= \begin{bmatrix}
0,& -\omega_{t,3},& \omega_{t,2} \\
\omega_{t,3},& 0,& -\omega_{t,1}  \\
-\omega_{t,2},& \omega_{t,1},& 0\\
\end{bmatrix}
$$. 


If $S_t$ is a constant, we can solve this differentiation equation. **But when can $S_t$ be a constant?** Suppose a rigid body rotate around a normalized $\phi$ constantly with rotation speed as $\|\phi\|$, then we can have:
$$
\begin{align}
R_t \omega &=  \phi \\
d\frac{R_t}{dt}\phi + R_t d\frac{\phi}{dt} &= 0 \\
d\frac{R_t}{dt}\phi &= 0 \\
S_tR_t\phi &= 0 \\
S_t \phi &= 0 \\
\begin{bmatrix}
0,& -\omega_{t,3},& \omega_{t,2} \\
\omega_{t,3},& 0,& -\omega_{t,1}  \\
-\omega_{t,2},& \omega_{t,1},& 0\\
\end{bmatrix} \begin{bmatrix}
\phi_0 \\
\phi_1 \\
\phi_2\\
\end{bmatrix} &= 0
\end{align}
$$
In order to make the $S_t \phi = 0$ always true, $\omega$ should be the same as $\phi$ so $[\omega]_\times \phi = 0$.

Now we can assume we have a constant $S_t$ as $S$ when a rigid body rotate constantly around a vector $\omega$, we can get
$$
\begin{align}
S_t &= [\omega]_\times \\
&=\begin{bmatrix}
0,& -\omega_3,& \omega_2 \\
\omega_3,& 0,& -\omega_1  \\
-\omega_2,& \omega_1,& 0\\
\end{bmatrix}
\end{align}
$$

By solving $d\frac{R_t}{dt} = SR_t$, we get
$$
\begin{align}
R_t &= R_0exp([\omega]_\times t) \\
\end{align}
$$

**Now we can have two very interesting conclusions:**
- If $R_0 = \mathcal{I}$, then $R_t = exp([\omega]_\times t)$. This means we can map a rotation axis $\omega$ to a rotation matrix with the exponential operation. This is actually exactly same as Rodrigues' rotation formula $exp([\omega]_\times) = R = \mathcal{I} + sin(\|\omega\|)[\omega]_\times + (1-cos(\|\omega\|))[\omega]_\times^2$. When $\omega$ is small, we should use Taylor expansion to get R or use an approximation $R \approx \mathcal{I} + [\omega]_\times$
- If $\omega$ is very small, then we can think $exp([\omega]_\times)$ is a small perturbation on the current state $R_0$. This is extremely useful for solving an optimization problem on SO(3) using iterative methods. 

### Introduction to $\boxplus$
Suppose the current state is $\bar{x}$, for approaches like Gauss-Newton, we normally would minimize the cost function $f$ as below. We assume that $\bar{x}$ and its perturbation $\Delta x$ in the same Euclidean space so that they be summed together use $+$.

$$
\begin{align}
\Delta  \hat{x} &= \underset{\Delta x}{\operatorname{argmin}}{f(\bar{x}+\Delta x)} 
\end{align}
$$

But what is a more general expression for this problem? For example, we have rotation matrix $R \in \mathbb{R^9}$ and its perturbation $\omega \in \mathbb{R^3}$. Clearly $+$ wouldn't work in this case. Thus we introduce a new plus here as $\boxplus$:
$$
\begin{align}
\Delta  \hat{x} &= \underset{\Delta x}{\operatorname{argmin}}{f(\bar{x} \boxplus \Delta x)} 
\end{align}
$$

For rotation:
$$
\begin{align}
\bar{R} \boxplus \Delta \omega =  \bar{R}exp([\omega]_\times)
\end{align}
$$
For translation:
$$
\begin{align}
\bar{t} \boxplus \Delta t =  \bar{t} + \Delta t
\end{align}
$$

### Jacobians

For the special case, we can find Jacobian matrix $J^{f(x)}_{x}|_{x = 0}$ to approximate $f$ to be a linear function, then solve the minimization problem. 
$$
\begin{align}
\Delta  \hat{x} &= \underset{\Delta x}{\operatorname{argmin}}{f(\bar{x}+\Delta x)}  \\
&\approx \underset{\Delta x}{\operatorname{argmin}}[f(\bar{x})+ J^{f(x)}_{x}|_{x = 0}\Delta x]
\end{align}
$$

For the general case, it becomes:
$$
\begin{align}
\Delta  \hat{x} &= \underset{\Delta x}{\operatorname{argmin}}{f(\bar{x} \boxplus \Delta x)}  \\
&\approx \underset{\Delta x}{\operatorname{argmin}}[f(\bar{x})+ J^{f(x\boxplus \Delta x)}_{x}|_{x = \bar{x}, \Delta x = 0}J^{\bar{x}\boxplus  x}_{x}|_{x = 0}\Delta x]
\end{align}
$$

We actually leverage chain rule to find proper Jacobian matrix as $J^{f(x\boxplus \Delta x)}_{x}|_{x = \bar{x}, \Delta x = 0}J^{\bar{x}\boxplus  x}_{x}|_{x = 0}$. 
- Though the first part seems complicated, $J^{f(x\boxplus \Delta x)}_{x}|_{x = \bar{x}, \Delta x = 0}$ is actually just $J^{f(x)}_{x}|_{x = 0}$/
- For the second part, if $\bar{x}$ and perturbation state $x$ in the same Euclidean space, then $J^{\bar{x}\boxplus  x}_{x}|_{x = 0} = \mathcal{I}$. Otherwise we have to calculate it, e.g. $J^{\bar{R}\boxplus  \omega}_{\omega}|_{\omega = 0} = J^{\bar{R}exp([\omega]_\times)}_{\omega}|_{\omega = 0} = []_{3 \times 1}$

Calculating Jacobian is not hard based on chain rules but is very tedious and error-prone. So using auto-differentiation would be a good idea. However, in order to get them analytically, we need know more about Lie theory which will be introduced throughly in the next section.

### State Correction

The state correction would be as simple as below:
$$
\begin{align}
x = \bar{x} \boxplus \Delta  \hat{x} 
\end{align}
$$

## More about Lie Theory for State Estimation in Robotics

$$
\begin{align}
Exp(\phi) :=\ & exp(\phi^{\wedge}) = R \\
Log(R) :=\ & log(R)^{\vee} = \phi \\
% \vee \wedge
\end{align}
$$

First-order approximation:
$$
\begin{align}
Exp(\phi + \delta\phi) &\approx Exp(\phi)Exp(J_r(\phi)\delta\phi) \\
Exp(\phi + J_r^{-1}(\phi)\delta\phi) &\approx Exp(\phi)Exp(J_r(\phi)J_r^{-1}(\phi)\delta\phi) \\
Log(Exp(\phi)Exp(\delta\phi)) &\approx \phi + J_r^{-1}(\phi)\delta\phi
\end{align}
$$

$$
\begin{align}
Exp(\phi) R = R Exp(R^T \phi) 
\end{align}
$$

Reference:
- Sola et al. `A micro Lie theory for state estimation in robotics`